In [91]:
import os,sys,re
import networkx as nx
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
#plt.xkcd()
import random
from datetime  import datetime,timedelta
import itertools
# from sympy import *
from more_itertools import chunked
# print(sys.path)
from PIL import Image
# os.chdir("/Users/miyachi.kei/workspace/work")
from bs4 import BeautifulSoup


%pylab inline --no-import-all

Populating the interactive namespace from numpy and matplotlib


In [92]:
RATE_YURI =5.2
RATE_FURI =4.8

In [93]:
names = {
"urn": "urien",
"veg": "vega",
"dsm": "dalsim",
"hnd": "honda",
"blk": "blanka",
"bsn": "bison",
"blr": "balrog",
"fan": "fang",
"brd": "bardy",
"krn": "karin",
"flk": "falke",
"abg": "abigail",
"gul": "guile",
"alx": "alex",
"ed":  "ed",
"g":   "g",
"kln": "kolin",
"cdy": "cody",
"zgf": "zangief",
"ibk": "ibuki",
"zku": "zeku",
"ncl": "necali",
"lar": "laura",
"nsh": "nash",
"rmk": "mika",
"ken": "ken",
"rsd": "rashid",
"psn": "poison",
"gok": "gouki",
"cnl": "chunli",
"ryu": "ryu",
"sgt": "sagat",
"jri": "juri",
"cmy": "cammy",
"skr": "sakura",
"mnt": "menat",
"kge": "kage",
"lca": "lucia",
"gil": "gill",
}

In [94]:
# html = None
# with open("./stats.html") as f:
#     html = f.read()
#     # 手作業でやったがクリーニングをここでやる


# for f,t in names.items():
#     html = html.replace(f,t)
pd.options.display.float_format = '{:.4f}'.format


In [95]:
import requests
url="https://game.capcom.com/cfn/sfv/stats/dia/"
r = requests.get(url)
raw_html = r.text


In [96]:
def get_icons(soup):
    import re
    images = {}
    for x in soup.find_all('img'):
        m = re.search("(https.*\/s\/)([a-zA-Z]+)(\.png\?h=[a-zA-Z0-9]+)", x['src'])
        if m:
            a,b,c = m.groups()
            images[names[b]] = a+b+c

    return images        

def create_clean_table(soup):
    # 不要な要素を削除
    for p in soup.select("p.total"):
        p.decompose()

    # imgタグのある箇所を名前に変更
    for x in soup.find_all("figure"):
        m = re.search("https.*\/s\/([a-zA-Z]+)\.png\?h=", x.find("img")["src"])
        if m:
            a = m.group(1)
            x.parent.string = a
    # "-" -> 0
    for x in soup.select("td.off"):
        x.string="0"
            
    html = str(soup)
    for f,t in names.items():
        html = html.replace(f,t)
    
    return html

def get_period(html):
    soup = BeautifulSoup(html, "lxml")
    return soup.select_one("p.selectData").string

period = get_period(raw_html)
soup = BeautifulSoup(raw_html, "lxml").table
images = get_icons(soup)
html = create_clean_table(soup)


In [97]:
df = pd.io.html.read_html(html)[0]
# column name change
df = df.rename(columns={'平均': 'name'})
df = df.set_index('name')
# 値をfloatに揃える
df = df.replace("-",0.0)
df = df.astype('float64')


In [98]:
df.head()

,honda,vega,urien,balrog,dalsim,blanka,fang,bison,bardy,abigail,...,juri,lucia,mika,gouki,sakura,cammy,menat,kage,rashid,gill
name,,,,,,,,,,,,,,,,,,,,,
honda,0.0000,5.0590,4.9620,4.9330,6.0770,5.6250,4.7870,5.8240,5.1970,4.8240,...,6.1640,5.9100,5.6560,5.5150,5.6110,6.2060,6.1890,5.7800,6.0900,6.5590
vega,4.9410,0.0000,5.0020,4.7280,5.2060,4.7960,5.3220,5.4610,5.0800,5.1660,...,5.8110,5.9050,5.3020,5.7120,5.8360,5.7340,6.2850,5.9950,5.6080,6.5210
urien,5.0380,4.9980,0.0000,5.0620,5.2600,5.2710,4.7140,5.1970,5.5530,5.4970,...,5.6920,5.7630,5.5850,5.5740,5.8830,5.8040,5.8960,5.9340,5.7210,6.3250
balrog,5.0670,5.2720,4.9380,0.0000,5.7220,5.1440,5.2770,4.9650,5.0250,5.9840,...,5.8200,5.9060,5.5690,5.5600,5.6230,5.6070,6.1230,5.9320,5.9470,6.5270
dalsim,3.9230,4.7940,4.7400,4.2780,0.0000,4.6350,4.9130,4.5760,5.2330,5.0030,...,5.3350,5.6440,5.4310,5.8530,5.9370,5.4800,5.7470,5.6690,5.1360,6.1760


In [99]:
# ダイアグラムの有利と不利のリストを作る
dia = {}
for k ,v in df.iterrows():
    # 名前の列を除外
    tmp = v[1:]
    # 0（自分）を除外
    yuri = tmp[(tmp!=0)&(tmp>RATE_YURI)] # 有利という以上は50%じゃ困る
    yuri = yuri.sort_values(ascending=False)
    furi = tmp[(tmp!=0)&(tmp<RATE_FURI)] # 49%切るなら不利かなという気持ち
    furi = furi.sort_values()
    dia[k] = {
        "yuri": yuri,
        "furi": furi,
    }



In [100]:
def create_recommend(df, dia, my):
    recommend = {}
    furi = set(dia[my]["furi"].keys())
    print(furi)
    for name, d in dia.items():
        tmp = furi & set(d["yuri"].keys())
        if not tmp:
            continue
        
        for t in tmp:
            recommend[name] = {t:df.loc[name,t] for t in tmp} # tmp # df.loc[name,t]
    return recommend

In [101]:
yuri = df[(df!=0)&(df>RATE_YURI)] # 有利という以上は50%じゃ困る
furi = df[(df!=0)&(df<RATE_FURI)] # 49%切るなら不利かなという気持ち

In [102]:
# いっさい該当ない行を除外
# yuri = yuri.dropna(how='all')
# furi = furi.dropna(how='all')

In [103]:
df.head()

,honda,vega,urien,balrog,dalsim,blanka,fang,bison,bardy,abigail,...,juri,lucia,mika,gouki,sakura,cammy,menat,kage,rashid,gill
name,,,,,,,,,,,,,,,,,,,,,
honda,0.0000,5.0590,4.9620,4.9330,6.0770,5.6250,4.7870,5.8240,5.1970,4.8240,...,6.1640,5.9100,5.6560,5.5150,5.6110,6.2060,6.1890,5.7800,6.0900,6.5590
vega,4.9410,0.0000,5.0020,4.7280,5.2060,4.7960,5.3220,5.4610,5.0800,5.1660,...,5.8110,5.9050,5.3020,5.7120,5.8360,5.7340,6.2850,5.9950,5.6080,6.5210
urien,5.0380,4.9980,0.0000,5.0620,5.2600,5.2710,4.7140,5.1970,5.5530,5.4970,...,5.6920,5.7630,5.5850,5.5740,5.8830,5.8040,5.8960,5.9340,5.7210,6.3250
balrog,5.0670,5.2720,4.9380,0.0000,5.7220,5.1440,5.2770,4.9650,5.0250,5.9840,...,5.8200,5.9060,5.5690,5.5600,5.6230,5.6070,6.1230,5.9320,5.9470,6.5270
dalsim,3.9230,4.7940,4.7400,4.2780,0.0000,4.6350,4.9130,4.5760,5.2330,5.0030,...,5.3350,5.6440,5.4310,5.8530,5.9370,5.4800,5.7470,5.6690,5.1360,6.1760


In [104]:
yuri

,honda,vega,urien,balrog,dalsim,blanka,fang,bison,bardy,abigail,...,juri,lucia,mika,gouki,sakura,cammy,menat,kage,rashid,gill
name,,,,,,,,,,,,,,,,,,,,,
honda,nan,nan,nan,nan,6.0770,5.6250,nan,5.8240,nan,nan,...,6.1640,5.9100,5.6560,5.5150,5.6110,6.2060,6.1890,5.7800,6.0900,6.5590
vega,nan,nan,nan,nan,5.2060,nan,5.3220,5.4610,nan,nan,...,5.8110,5.9050,5.3020,5.7120,5.8360,5.7340,6.2850,5.9950,5.6080,6.5210
urien,nan,nan,nan,nan,5.2600,5.2710,nan,nan,5.5530,5.4970,...,5.6920,5.7630,5.5850,5.5740,5.8830,5.8040,5.8960,5.9340,5.7210,6.3250
balrog,nan,5.2720,nan,nan,5.7220,nan,5.2770,nan,nan,5.9840,...,5.8200,5.9060,5.5690,5.5600,5.6230,5.6070,6.1230,5.9320,5.9470,6.5270
dalsim,nan,nan,nan,nan,nan,nan,nan,nan,5.2330,nan,...,5.3350,5.6440,5.4310,5.8530,5.9370,5.4800,5.7470,5.6690,nan,6.1760
blanka,nan,5.2040,nan,nan,5.3650,nan,nan,5.2660,nan,nan,...,5.5280,5.4640,5.5010,5.7000,5.6080,5.5830,6.2410,5.7910,5.6430,6.4380
fang,5.2130,nan,5.2860,nan,nan,nan,nan,5.3110,nan,5.2290,...,5.5270,5.5900,5.5170,5.4880,5.6130,5.4050,5.8370,5.4720,nan,6.5200
bison,nan,nan,nan,nan,5.4240,nan,nan,nan,nan,nan,...,5.5840,5.7690,5.3820,5.4340,5.6950,5.6410,5.9190,5.6470,5.7440,6.2180
bardy,nan,nan,nan,nan,nan,nan,nan,5.2770,nan,nan,...,5.5520,5.3980,5.3770,5.4220,5.2740,5.5550,5.3850,5.7760,5.3160,5.9230


In [105]:
furi

,honda,vega,urien,balrog,dalsim,blanka,fang,bison,bardy,abigail,...,juri,lucia,mika,gouki,sakura,cammy,menat,kage,rashid,gill
name,,,,,,,,,,,,,,,,,,,,,
honda,nan,nan,nan,nan,nan,nan,4.7870,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
vega,nan,nan,nan,4.7280,nan,4.7960,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
urien,nan,nan,nan,nan,nan,nan,4.7140,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
balrog,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
dalsim,3.9230,4.7940,4.7400,4.2780,nan,4.6350,nan,4.5760,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
blanka,4.3750,nan,4.7290,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
fang,nan,4.6780,nan,4.7230,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
bison,4.1760,4.5390,nan,nan,nan,4.7340,4.6890,nan,4.7230,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
bardy,nan,nan,4.4470,nan,4.7670,nan,nan,nan,nan,4.6500,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [106]:
def beautify(html, delete_name=False):
    soup = BeautifulSoup(html, "lxml")
    for tag in soup():
        for attribute in ["class", "id", "name", "style","border"]:
            del tag[attribute]
    # css kit 用のクラスを割り付ける
    t = soup.find("table")
    t["class"] = "uk-table uk-table-divider  uk-table-small"
    # name だけある行が不要なので消す
    tr = soup.find("tr")
    if delete_name:
        soup("tr")[1].decompose()
    # todo: 最後にhtmlとbodyたぐを除去する
    return str(soup.table)

In [107]:
f  = open(f"../stats.html", "w", encoding="utf-8")
f.write(f"""<!DOCTYPE html>
<html>
<head>
<meta charset="UTF-8">
<title>Disadvantages and Recommend({period}) | Street fighter V Champion Edition</title>
<!-- UIkit CSS -->
<link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/uikit@3.2.6/dist/css/uikit.min.css" />

<!-- UIkit JS -->
<script src="https://cdn.jsdelivr.net/npm/uikit@3.2.6/dist/js/uikit.min.js"></script>
<script src="https://cdn.jsdelivr.net/npm/uikit@3.2.6/dist/js/uikit-icons.min.js"></script>
</head>

<body>
""")
f.write('<header class="uk-section uk-section-primary uk-light  uk-padding">')
f.write('<h1>Disadvantages and Recommend</h1>')
f.write("</header>")

f.write("""<div class="uk-section uk-section-default uk-padding">""")
f.write(f"""<p>自分のキャラを選ぶと、下に不利な組み合わせと、それに有効なかぶせを表示します。<br>
集計期間: {period}
</p>""")


# ----------

f.write('<div class="uk-subnav uk-subnav-pill uk-flex" uk-switcher>')
for idx, x in enumerate(df.columns):
    f.write(f"""<div><a href="#{x}" uk-switcher-item="{idx}">{x.capitalize()}</a></div>""")
f.write("</div>")


# -------------------

f.write("""<div class="uk-switcher uk-margin">""")
idx = 0
for k,fr in furi.iterrows():
    f.write(f"""<div uk-switcher-item="{idx}">""")

    ks = [k for k in fr.dropna().keys()]
    yr = yuri[list(ks)].dropna(how='all')#.dropna(how='all', axis=1)
    yr = yr.replace(np.nan, '', regex=True)
    html = yr.to_html()

    f.write(f"""<h3 class="uk-heading-medium uk-heading-line uk-text-center"><a name="{k}">{k.capitalize()}</a></h3>\n""")
    f.write("""<h4 class="uk-heading-small">Disadvantages:</h4>""")

    f.write("""<div class="uk-overflow-auto">""")

    fr = pd.DataFrame(furi.loc[k].dropna(how='all'))
    if len(fr):
        # 表示用には転置
        f.write(beautify(fr.T.to_html()))
    else:
        f.write("<p>Not disadvantaged for everything.</p>")
        
    f.write("""</div>""")

    
    f.write("""<h4 class="uk-heading-small">Counter pics recommend:</h4>""")
    # pandasのhtmlをきれいにする
    f.write("""<div class="uk-overflow-auto">""")
    f.write(beautify(html, False))
    f.write("""</div>""")
    f.write("""</div>""")
    idx += 1
    
f.write("""</div>""")

    
# padding
f.write(f"""</div>
<div class="uk-section uk-section-muted uk-padding uk-text-small">
シャドルー研究所のstats抜き出して、統計上の有利（{RATE_YURI}以上）/不利（{RATE_FURI}以下）を分けて出しているだけのものです。<br>
たぶん使用率の低いキャラ・追加間もないキャラの数値は特に動きやすいです。<br>
それサブで出します？みたいな例もありますが一種の冗談と思って見るくらいでお願いします。
</div>

<footer class="uk-section uk-section-secondary uk-light  uk-padding">
author:　<a href="https://twitter.com/schaft_aurea">Halipeco</a><br>
The original data is on <a href="https://game.capcom.com/cfn/sfv/stats/dia/">対戦ダイヤグラム | 統計情報 | CAPCOM：シャドルー格闘家研究所</a><br>
©2020 Valve Corporation. Steam and the Steam logo are trademarks and/or registered trademarks of Valve Corporation in the U.S. and/or other countries.<br>
©CAPCOM U.S.A., INC. 2016, 2018 ALL RIGHTS RESERVED.
</footer>
</body>
</html>""")

f.close()

In [108]:

len(s)

NameError: name 's' is not defined